In [6]:
# Konstanten, welche angepasst werden können
## Gibt an in wie viele Gruppen die Geschwindigkeit eingeteilt werden soll
GESCHWINDIGKEIT_BINS = 5
## Gibt an in wieviele Gruppen die Herzfrequenz eingeteilt werden soll
HERZFREQUENZ_BINS = 5

In [7]:
import fitparse
import os
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
import pandas as pd
import numpy as np
from fit_extractor import extract_day, extract_distance, extract_avg_speed, extract_avg_hf, groupInBins


directory_path = 'data/v2/'

# Lese alle im Verzeichnis vorliegenden Dateien ein
fitfiles = []
for file in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file)
    if os.path.isfile(file_path):
        fitfiles.append(fitparse.FitFile(file_path))

In [8]:
# Ermittle interessante Daten zu jeder eingelesenen Datei
dates = []
distances = []
avg_speeds = []
avg_hf = []
for file in fitfiles:
    dates.append(extract_day(file))
    distances.append(extract_distance(file))
    avg_speeds.append(extract_avg_speed(file))
    avg_hf.append(extract_avg_hf(file))

# Speichere Daten in einem Dataframe und sortiere diesen
df = pd.DataFrame({'Datum':dates, 'Distanz in km':distances, 'Ø Geschwindigkeit in km/h':avg_speeds, 'Ø Herzfrequenz in bpm': avg_hf})
df.sort_values(by='Datum', inplace=True)

In [9]:
# Transformation und Gruppierung der Daten
binnedDf = df.copy()
binnedDf['Distanz in km'] = df['Distanz in km'].apply(lambda x: round(x/1000))
binnedDf['Ø Geschwindigkeit in km/h'] = df['Ø Geschwindigkeit in km/h'].apply(lambda x: groupInBins(df['Ø Geschwindigkeit in km/h'], x, GESCHWINDIGKEIT_BINS))
binnedDf['Ø Herzfrequenz in bpm'] = df['Ø Herzfrequenz in bpm'].apply(lambda x: groupInBins(df['Ø Herzfrequenz in bpm'], x, HERZFREQUENZ_BINS))

In [10]:
categorical_dimensions = ['Distanz in km', 'Ø Geschwindigkeit in km/h', 'Ø Herzfrequenz in bpm'];
dimensions = [dict(values=binnedDf[label], label=label) for label in categorical_dimensions]

# Farbverlauf definieren
color = np.zeros(len(binnedDf), dtype='uint8')
colorscale = [[0, 'gray'], [1, 'firebrick']]

# Visualisierung definieren
fig = go.FigureWidget(
    data=[go.Scatter(x=binnedDf.Datum, y=binnedDf['Distanz in km'],
    marker={'color': 'gray'}, mode='markers', selected={'marker': {'color': 'firebrick'}},
    unselected={'marker': {'opacity': 0.3}}), go.Parcats(
        domain={'y': [0, 0.4]}, dimensions=dimensions,
        line={'colorscale': colorscale, 'cmin': 0,
              'cmax': 1, 'color': color, 'shape': 'hspline'})
    ])

fig.update_layout(
        height=800, xaxis={'title': 'Zeit'},
        yaxis={'title': 'Distanz in km', 'domain': [0.6, 1]},
        dragmode='lasso', hovermode='closest')

# Interaktionen definieren
def update_color(trace, points, state):
    fig.data[0].selectedpoints = points.point_inds
    new_color = np.zeros(len(df), dtype='uint8')
    new_color[points.point_inds] = 1
    fig.data[1].line.color = new_color

fig.data[0].on_selection(update_color)
fig.data[1].on_click(update_color)

fig

FigureWidget({
    'data': [{'marker': {'color': 'gray'},
              'mode': 'markers',
              'sele…